In [ ]:
# load the model from the file "sbm3x3_pclam_roc_0.210_auc_0.860"
import torch
from torch_geometric.transforms import TwoHop

import sys
if '..' not in sys.path:
    sys.path.insert(0, '..')

from datasets.import_dataset import import_dataset
from utils.plotting import *
from trainer import Trainer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Using device:', device)
cpu = torch.device("cpu")

%load_ext autoreload
%autoreload 2


In [ ]:

model_names = ['piegam', 'iegam', 'pclam', 'bigclam']
ds_names = ['ellipticGGAD', 'redditGGAD', 'photoGGAD']
#! problem with something to do with attr with reddit and elliptic
for ds_name in ds_names:    
    ds = import_dataset(ds_name)
    fat_ds = TwoHop()(ds)
    fat_ds.edge_attr = torch.ones(fat_ds.edge_index.shape[1]).bool()

    if ds_name in ['Flickr', 'ACM', 'BlogCatalog']:
        ds_to_use = ds
    elif ds_name in ['redditGGAD', 'photoGGAD', 'ellipticGGAD']:
        ds_to_use = fat_ds
    else:
        raise ValueError('ds_name not recognized')

    losseses = []
    acc_testses = []
    acc_valses = []
    
    for model_name in model_names:
        '''change some of the configs manually e.g. 
        config_triplets = [['feat_opt', 'n_iter', 1000], ['prior_opt, 'lr', 0.0001], ...]'''
        config_triplets = []

        trainer_anomaly = Trainer(
            model_name=model_name,
            device=device,
            dataset=ds_to_use.clone(),
            attr_opt=True,
            task='anomaly',
            mighty_configs_dict=True,
            config_triplets_to_change=config_triplets
        )

        losses, acc_test, acc_val = trainer_anomaly.train(
            init_type='small_gaus',
            init_feats=True,
            acc_every=20,
            plot_every=-1,
            verbose=True,
            verbose_in_funcs=False
        )
        losseses.append(losses)
        acc_testses.append(acc_test)
        acc_valses.append(acc_val)
        
        if trainer_anomaly.clamiter.prior is not None:
            del trainer_anomaly.clamiter.prior.model
        del trainer_anomaly.data


